**need some %run command and uncomment the model load before submitting**

In [1]:
import sys,json
from typing import Callable, Dict, List, Set, Tuple
from elasticsearch import Elasticsearch
import gensim.downloader as api
import gensim
import datetime
sys.path.insert(0, 'extract_features')
from extract_No1to5_features import extract_features_1to5
from extract_No11to12_features import extract_features_11to12
from extract_No13to15_features import TypeTaxonomy, extract_features_13to15
from extract_No16_feature import extract_features_16
from extract_No17to19_features import get_analyze,extract_features_17to19
from extract_No20to22_features import extract_features_20to22
from extract_No23to25_features_optimized import extract_features_23to25

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\junec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\junec\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# Reduce logging level.
import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.WARNING)
logging.basicConfig(level=logging.ERROR)

logging.getLogger("elasticsearch").disabled = True
# es_log = logging.getLogger("elasticsearch")
# es_log.setLevel(logging.CRITICAL)
es= Elasticsearch()
es.info()

{'name': 'LAPTOP-ADBLIUPR',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '5VELmBwJTk-urTuhZdTgew',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'zip',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [3]:
#%run extract_features/extract_No11to12_features

**for my computer , need the following enviroment to run this script**
<br>
/c/Users/junec/anaconda3/python
<br>
/c/Users/junec/anaconda3/Scripts/pip

In [4]:
!which python
!which pip

/c/Users/junec/anaconda3/python
/c/Users/junec/anaconda3/Scripts/pip


In [5]:
#load model
try:
    model_loaded = gensim.models.keyedvectors.KeyedVectors.load('googleNews.d2v')
except:
    model_loaded = api.load('word2vec-google-news-300')
    model_loaded.save('googleNews.d2v')
    model_loaded = gensim.models.keyedvectors.KeyedVectors.load('googleNews.d2v')

2021-11-19 23:49:43,437 : INFO : loading KeyedVectors object from googleNews.d2v
2021-11-19 23:49:46,252 : INFO : loading vectors from googleNews.d2v.vectors.npy with mmap=None
2021-11-19 23:51:31,867 : INFO : KeyedVectors lifecycle event {'fname': 'googleNews.d2v', 'datetime': '2021-11-19T23:51:29.636261', 'gensim': '4.1.2', 'python': '3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'loaded'}


In [ ]:
#load files
filepath="data/DBpedia_map_type_entities.json"
with open(filepath, 'r', encoding='utf-8') as file:
    DBpedia_map_type_entities = json.load(file)
    
filepath="data/training_types.json"
with open(filepath,encoding='utf-8') as json_file:
    training_map_type_questions = json.load(json_file)

filepath="data/ElasticSearch_map_type_docID.json"
with open(filepath, 'r',encoding='utf-8') as f:
    docID_DBOtype_dict = json.load(f)

filepath="../smart-dataset/datasets/DBpedia/smarttask_dbpedia_train.json"
with open(filepath, 'r') as f:
    smarttask_dbpedia_train = json.load(f)

typeobj=TypeTaxonomy("data/dbpedia_types.tsv")

In [ ]:
# type_list=list(typeobj._types.keys())
# type_list[1:]
# len(type_list[1:]),len(smarttask_dbpedia_train) ,smarttask_dbpedia_train[1]

In [ ]:
# dp_type="dbo:MilitaryPerson"
# question="Does the shelf life of spinach equal 8?"

# dp_type="dbo:MusicFestival"
# question="When was Bibi Andersson music festival married to Per Ahlmark very green?"

# extract_features_1to5(DBpedia_map_type_entities,dp_type,question,es)
# features_1to5=extract_features_1to5(DBpedia_map_type_entities,dp_type,question,es)
# extract_features_11to12(dp_type, question,docID_DBOtype_dict,es)
# extract_features_13to15(typeobj,dp_type)
# print(extract_features_16(DBpedia_map_type_entities,dp_type))

# extract_features_17to19(training_map_type_questions,dp_type,question)
# extract_features_20to22(dp_type,question)
# extract_features_23to25(model_loaded,dp_type, question, mode="Euclidean")
# extract_features_23to25(model_loaded,dp_type, question, mode="similarities")

In [ ]:
def map_type_featureVectors_13to16(typeobj:TypeTaxonomy,
                        DBpedia_map_type_entities:Dict
                        )->List[Dict[str,List]]:
    
    map_type_featureVectors={}
    all_DBOtype=list(typeobj._types.keys())
    #remove the first type:"thing"
    all_DBOtype=all_DBOtype[1:]
    for DBOtype in all_DBOtype:
        dict_13to16={}
        features_13to15 = extract_features_13to15(typeobj,DBOtype)
        feature_vect =list(features_13to15.values())

        features_16=extract_features_16(DBpedia_map_type_entities,DBOtype)
        feature_vect.extend(list(features_16.values()))
     
        map_type_featureVectors[DBOtype]=feature_vect
        
    return map_type_featureVectors
        

# features_13to16=map_type_featureVectors_13to16(typeobj,DBpedia_map_type_entities)    

In [ ]:
def extract_features(
    question:str,
    dp_type: str,
    DBpedia_map_type_entities:Dict,
    docID_DBOtype_dict:Dict,
    typeobj:TypeTaxonomy,
    training_map_type_questions:Dict,
    model_loaded:gensim.models.keyedvectors.KeyedVectors,
    es: Elasticsearch,
    analyze,X,terms_corpus,
    features_13to16:List[Dict[str,List]],
    add_extra_features:bool=False
) -> List[float]:
    """Extracts features of a query and document pair.

        Args:
            query: string.
            dp_type: DBO type.
            es: Elasticsearch object instance.

        Returns:
            List of extracted feature values in a fixed order.
    """
    features_1to5 = extract_features_1to5(DBpedia_map_type_entities,dp_type,question,es)
    feature_vect = list(features_1to5.values())

    features_11to12 = extract_features_11to12(dp_type, question,docID_DBOtype_dict,es)
    feature_vect.extend(list(features_11to12.values()))

#     features_13to15 = extract_features_13to15(typeobj,dp_type)
#     feature_vect.extend(list(features_13to15.values()))
    
#     features_16=extract_features_16(DBpedia_map_type_entities,dp_type)
#     feature_vect.extend(list(features_16.values()))
    feature_vect.extend(features_13to16[dp_type])
    
   
    features_17to19=extract_features_17to19(analyze,X,terms_corpus,training_map_type_questions,dp_type,question)
    feature_vect.extend(list(features_17to19.values()))
    
    features_20to22=extract_features_20to22(dp_type,question)
    feature_vect.extend(list(features_20to22.values()))
    
    features_23to25=extract_features_23to25(model_loaded,dp_type, question, mode="Euclidean")
    feature_vect.extend(list(features_23to25.values()))

    if add_extra_features:
        features_23to25_variant=extract_features_23to25(model_loaded,dp_type, question, mode="similarities")
        feature_vect.extend(list(features_23to25_variant.values()))
    
    return feature_vect

In [ ]:
# analyze,X,terms_corpus=get_analyze(training_map_type_questions)
# dp_type="dbo:MusicFestival"
# question="When was Bibi Andersson music festival married to Per Ahlmark very green?"
# features= extract_features(
#                     question,
#                     dp_type,
#                     DBpedia_map_type_entities,
#                     docID_DBOtype_dict,
#                     typeobj,
#                     training_map_type_questions,
#                     model_loaded,
#                     es,
#                     analyze,X,terms_corpus,
#                     features_13to16) 

In [ ]:
def prepare_training_data(  smarttask_dbpedia_train:List[Dict],
                            DBpedia_map_type_entities:Dict[str,List],
                            docID_DBOtype_dict:Dict[str,str],
                            typeobj:TypeTaxonomy,
                            training_map_type_questions:Dict[str,str],
                            model_loaded:gensim.models.keyedvectors.KeyedVectors,
                            es: Elasticsearch,
                            add_extra_features:bool=False)-> Tuple[List[List[float]], List[int]]:
    """
    """
    X_train=[]
    y_train=[]
    all_DBOtype=list(typeobj._types.keys())
    #remove the first type:"thing"
    all_DBOtype=all_DBOtype[1:]
    analyze,X,terms_corpus=get_analyze(training_map_type_questions)
    features_13to16=map_type_featureVectors_13to16(typeobj,DBpedia_map_type_entities)    
    count=0
    j=0
    for entry in smarttask_dbpedia_train:
#         if j%int(len(smarttask_dbpedia_train)/100)==0:
#             print(f'--------------{j/int(len(smarttask_dbpedia_train)/100)}% is finished')
        if j%100==0:
            print("--------------------j:",j)
        
        j+=1
        if entry['question']==None:
            continue
        if entry['category']=='resource':
            for DBOtype in all_DBOtype: 
                try:
                    features=extract_features(
                    entry['question'],
                    DBOtype,
                    DBpedia_map_type_entities,
                    docID_DBOtype_dict,
                    typeobj,
                    training_map_type_questions,
                    model_loaded,
                    es,
                    analyze,X,terms_corpus,
                    features_13to16) 
                except BaseException as err:
                    print("------------error for type:",DBOtype,entry['question'])
                    print(f"Unexpected {err}, {type(err)}")  
                    raise
                X_train.append(features)
                count+=1
                #print(f'--------------length of X_train:{len(X_train)} y_train: {len(y_train)}')
                if DBOtype in entry['type']: 
                    y_train.append(1)
                else:
                    y_train.append(0)
                    
                if count%100000==0:
                    print(f'--------------{count} is finished')
                if len(X_train)%500==0:
                    print("-----------------",datetime.datetime.now())
                    print(f'--------------length of X_train:{len(X_train)} y_train: {len(y_train)}')
        
    print(f'{count} features has been saved,{j} questions have been processed')
    return X_train,y_train


# with open("../data/training_types.json", 'w',encoding='utf-8') as f:
#   json.dump(collections, f, ensure_ascii=False)

X_train,y_train=prepare_training_data(smarttask_dbpedia_train,
                                        DBpedia_map_type_entities,
                                        docID_DBOtype_dict,
                                        typeobj,
                                        training_map_type_questions,
                                        model_loaded,
                                        es) 

In [ ]:
with open("../data/X_train.json", 'w') as f:
  json.dump(collections, f)
with open("../data/y_train.json", 'w') as f:
  json.dump(collections, f)
print("X_train,y_train are saved as json files")

In [ ]:
# es=Elasticsearch()
# es.get(index='dbpdiea_type_centric',id="3")

In [ ]:
# index='dbpdiea_type_centric'
# field="abstract"
# tv = es.termvectors(index=index, doc_type="_doc", id="1", fields=field, term_statistics=True) 
# collection_len=tv["term_vectors"][field]['field_statistics']['sum_ttf']  
# #total number of entity in the collections
# doc_number=tv["term_vectors"][field]['field_statistics']['doc_count'] 
# #average document length
# avgdl=collection_len/doc_number
# collection_len,doc_number,avgdl

In [ ]:
# word="theatre"
# #model_loaded[word]
# model_loaded.most_similar(positive=[word], topn=1)